# Análisis de la Correlación entre el S&P 500 y el VIX

Este Jupyter Notebook tiene como objetivo explorar la **correlación entre el índice S&P 500 y el índice VIX**. Para ello, analizaremos un conjunto de datos que incluye información diaria del S&P 500 desde el 1 de enero de 2005 hasta el 18 de julio de 2025.

---

## Datos a Utilizar

El dataset del S&P 500 contiene las siguientes columnas:

* **date**: Fecha de la observación.
* **open**: Precio de apertura.
* **high**: Precio máximo.
* **low**: Precio mínimo.
* **close**: Precio de cierre.
* **volume**: Volumen de negociación.
* **sp_pct_move**: Porcentaje de cambio diario del S&P 500.

Además de los datos del S&P 500, incorporaremos los mismos datos para el **índice VIX**, conocido como el "índice del miedo", ya que mide la volatilidad esperada del S&P 500.

---

## Objetivos del Análisis

El estudio se centrará en los siguientes puntos clave:

* **Encontrar correlaciones** entre el S&P 500 y el VIX.
* Identificar **patrones en extensiones de movimiento** de ambos índices.
* Calcular la **media de movimiento** del S&P 500 en torno a diferentes rangos de precios del VIX.
* Realizar **cálculos de desviación estándar** para entender la dispersión de los movimientos cuando se encuentran correlaciones.

---

In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Cargar el data set que se descargo previamente de yfinance con el ticker ES=F

In [84]:
ES_df = pd.read_csv("C:/Users/juana/OneDrive/Escritorio/Datos_del_mercado/sp500_data_daily.csv")
ES_df

,date,open,high,low,close,volume,sp_pct_move
0,2005-01-03,1213.50,1221.25,1201.75,1206.25,672360,1.606922
1,2005-01-04,1206.25,1209.50,1186.75,1191.00,869562,1.886010
2,2005-01-05,1190.75,1195.00,1182.75,1183.25,855689,1.028763
3,2005-01-06,1183.25,1193.75,1182.75,1188.25,678414,0.929643
4,2005-01-07,1188.50,1194.50,1183.50,1186.25,766197,0.925536
...,...,...,...,...,...,...,...
5171,2025-07-14,6274.00,6315.00,6259.75,6311.00,909223,0.880618
5172,2025-07-15,6309.50,6343.00,6272.50,6284.00,1279068,1.117363
5173,2025-07-16,6273.75,6308.75,6241.00,6303.25,1462969,1.079896
5174,2025-07-17,6299.25,6345.50,6288.25,6340.50,1052843,0.908838


## Cargar el data set que se descargo previamente de yfinance con el ticker ^VIX

In [87]:
VIX_df = pd.read_csv("C:/Users/juana/OneDrive/Escritorio/Datos_del_mercado/vix_data_daily.csv")
VIX_df

,date,open,high,low,close,volume,vix_avg_hl
0,2005-01-03,13.390000,14.230000,13.250000,14.080000,0,13.740000
1,2005-01-04,14.010000,14.450000,13.930000,13.980000,0,14.190000
2,2005-01-05,13.980000,14.090000,13.260000,14.090000,0,13.675000
3,2005-01-06,14.090000,14.090000,13.330000,13.580000,0,13.710000
4,2005-01-07,13.470000,13.510000,12.940000,13.490000,0,13.225000
...,...,...,...,...,...,...,...
5163,2025-07-14,17.730000,17.850000,16.910000,17.200001,0,17.380000
5164,2025-07-15,16.889999,17.389999,16.559999,17.379999,0,16.974999
5165,2025-07-16,17.660000,19.480000,16.690001,17.160000,0,18.085000
5166,2025-07-17,17.160000,17.370001,16.480000,16.520000,0,16.925000


---
## Corrección y Normalización de Datos

Como podemos observar al analizar nuestros DataFrames, notamos que el DataFrame del **ES (S&P 500)** tiene más filas que el DataFrame del **VIX**. Esta discrepancia podría deberse a varios factores:

* Un **error en la extracción de datos** desde `yfinance` para el VIX.
* La presencia de **valores nulos** en el DataFrame del VIX, que no se registraron adecuadamente.
* Posibles **días feriados** o no operativos en los que el VIX no registró datos específicos.

Para asegurar la coherencia y la manipulación adecuada de los datos para nuestro estudio, es fundamental que ambos DataFrames tengan la misma cantidad de filas y que estas correspondan a las mismas fechas. Por lo tanto, procederemos a identificar y eliminar las fechas que no concuerden entre el ES y el VIX, normalizando así ambos conjuntos de datos.

---

In [98]:
# Fechas únicas en cada df
sp500_dates = set(ES_df['date'])
vix_dates = set(VIX_df['date'])

# Días en sp500_df pero no en vix_df
solo_sp500 = sp500_dates - vix_dates
print("Fechas solo en SP500:", sorted(solo_sp500))

# Días en vix_df pero no en sp500_df
solo_vix = vix_dates - sp500_dates
print("Fechas solo en VIX:", sorted(solo_vix))



Fechas solo en SP500: ['2005-11-24', '2006-01-16', '2006-02-20', '2006-05-29', '2006-07-04', '2006-09-04', '2006-11-23', '2023-11-23', '2025-01-09', '2025-07-04']
Fechas solo en VIX: ['2016-10-10', '2016-11-11']


In [100]:
#Hacemos una lista de las fechas que no coinsiden en los data frame
lista_fechas = ['2005-11-24', '2006-01-16', '2006-02-20', '2006-05-29', 
'2006-07-04', '2006-09-04', '2006-11-23', '2023-11-23', '2025-01-09', '2025-07-04',
'2016-10-10', '2016-11-11']

#Ahora eliminamos las filas que contengan estas fechas en la columna 'date'

# El simbolo ~ es una negacion en el df para que los valores que no encuentre en la lista 
# de fechas que queremos eliminar las deje como TRUE y las que si encontro las deje como FALSE 
#Para eliminar las fechas que queremos
ES_df = ES_df[~ ES_df['date'].isin(lista_fechas)]
VIX_df = VIX_df[~ VIX_df['date'].isin(lista_fechas)]

#Reseteamos el indice del df ES y VIX
ES_df.reset_index(drop=True,inplace=True)
VIX_df.reset_index(drop=True,inplace=True)

print(ES_df.tail())
print(VIX_df.tail())

            date     open     high      low    close   volume  sp_pct_move
5161  2025-07-14  6274.00  6315.00  6259.75  6311.00   909223     0.880618
5162  2025-07-15  6309.50  6343.00  6272.50  6284.00  1279068     1.117363
5163  2025-07-16  6273.75  6308.75  6241.00  6303.25  1462969     1.079896
5164  2025-07-17  6299.25  6345.50  6288.25  6340.50  1052843     0.908838
5165  2025-07-18  6342.50  6357.00  6323.25  6334.75  1052843     0.532125
            date       open       high        low      close  volume  \
5161  2025-07-14  17.730000  17.850000  16.910000  17.200001       0   
5162  2025-07-15  16.889999  17.389999  16.559999  17.379999       0   
5163  2025-07-16  17.660000  19.480000  16.690001  17.160000       0   
5164  2025-07-17  17.160000  17.370001  16.480000  16.520000       0   
5165  2025-07-18  16.430000  16.809999  16.280001  16.410000       0   

      vix_avg_hl  
5161   17.380000  
5162   16.974999  
5163   18.085000  
5164   16.925000  
5165   16.545000  


---
## Análisis Comparativo y Estadístico

¡Perfecto! Ahora que hemos **normalizado los datos** en ambos DataFrames, asegurando que las fechas coincidan, estamos listos para iniciar un **análisis comparativo preciso**.

Podremos explorar la relación entre el S&P 500 y el VIX de diversas maneras, incluyendo:

* Comparar el **movimiento promedio del S&P 500** con la **media diaria del VIX**.
* Identificar **patrones de correlación** en extensiones de movimiento.
* Calcular **desviaciones estándar** para entender la variabilidad en diferentes rangos de VIX.

Este paso es crucial para garantizar que nuestras futuras comparaciones y cálculos sean válidos y nos permitan obtener conclusiones significativas sobre la dinámica entre estos dos importantes indicadores del mercado.

---

In [128]:
#Creamos un df con los datos que necesitamos para nuestro estudio

df = pd.DataFrame(columns=['es_date','sp_pct_move','vix_date','vix_avg_hl'])
df['es_date']=ES_df['date']
df['sp_pct_move'] = ES_df['sp_pct_move']
df['vix_date']= VIX_df['date']
df['vix_avg_hl']=VIX_df['vix_avg_hl']
#df.reset_index(inplace=True)
df

#df.sort_values(['vix_avg_hl'])


,es_date,sp_pct_move,vix_date,vix_avg_hl
0,2005-01-03,1.606922,2005-01-03,13.740000
1,2005-01-04,1.886010,2005-01-04,14.190000
2,2005-01-05,1.028763,2005-01-05,13.675000
3,2005-01-06,0.929643,2005-01-06,13.710000
4,2005-01-07,0.925536,2005-01-07,13.225000
...,...,...,...,...
5161,2025-07-14,0.880618,2025-07-14,17.380000
5162,2025-07-15,1.117363,2025-07-15,16.974999
5163,2025-07-16,1.079896,2025-07-16,18.085000
5164,2025-07-17,0.908838,2025-07-17,16.925000
